# Artificial Neural Network - Classfication

**cycle** (initialize parameters with some weights => Forward Propagation => Compute Error / Loss (Diff between predicted values and actual values) => Adjust Parameters (weights of parameters) / Backward Propagation)

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix, accuracy_score

2023-02-26 12:12:04.575068: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-02-26 12:12:04.608649: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-02-26 12:12:04.760808: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-02-26 12:12:04.761595: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-26 12:12:05.537013: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Co

In [2]:
tf.__version__

'2.12.0-rc0'

## Data Preprocessing

### Loading Dataset


In [5]:
dataset = pd.read_csv("../../datasets/bank_customers_details.csv", sep=",")
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [130]:
samples_count, features_count = dataset.shape
samples_count, features_count

(10000, 14)

### Separating target column from dataset


In [131]:
X = dataset.iloc[:, 3:-1].values # first three columns are useless so discard them
y = dataset.iloc[:, -1].values

### Encoding categorical data

#### Label Encoding the Gender Column

In [132]:
le = LabelEncoder()
X[:, 2] = le.fit_transform(X[:, 2])

#### One Hot Encoding Geography column

In [133]:
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

## Train & Test split

In [134]:
train_x, test_x, train_y, test_y = train_test_split(X, y, test_size=0.2, random_state=0)

## Feature Scaling

In [135]:
sc = StandardScaler()
train_x = sc.fit_transform(train_x)
test_x = sc.transform(test_x)

## Building the ANN
**activation** = activation function
relu = rectifier activation function

**units** = no of neurons (no rule of thumb to choose neurons)



**At output layer**
units should be = no of classes in target column
if there are only two classes then units = 1 (as we want to predict binay classes 0 / 1)
but let's suppose we want to predict 3 classes (A, B, C) then One Hot Encoding of these classes should be
1 0 0
0 1 0

0 0 1

so in that case we need 3 neurons so units should be = 3

use sigmoid activation function for output layer

### Initializing the ANN

In [136]:
ann = tf.keras.models.Sequential()

### Adding the input layer and the first hidden layer

In [137]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Adding the second hidden layer

In [138]:
ann.add(tf.keras.layers.Dense(units=6, activation='relu'))

### Adding the output layer


In [139]:
ann.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

## Training the ANN

### Compiling the ANN
**optimizer** is used to adjust weights of neurons when there is a difference between actual value and predicted value

**loss** is used to find difference between actual value and predicted value

**binary_crossentropy** is used for binary classification (when we have two classes to predict)

**categorical_crossentropy** is used for multiple classes classification (more than 2 classes to predict)

but in that case use softmax as activation function in output layer

In [140]:
ann.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

### Training the ANN on the training set

**batch_size** is used to mention no of predictions comparing to actual results to know the loss at the same time instead of comparing one by one which is a slow process. 32 is the default value and recommended as well

**epoch** followingcycle is called epoch

In [141]:
ann.fit(train_x, train_y, batch_size=32, epochs=100)

Epoch 1/100
250/250 [==============================] - 0s 1ms/step - loss: 0.5281 - accuracy: 0.7846
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4644 - accuracy: 0.7960
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4468 - accuracy: 0.7964
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4396 - accuracy: 0.8014
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4345 - accuracy: 0.8040
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4295 - accuracy: 0.8073
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4254 - accuracy: 0.8126
Epoch 8/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4214 - accuracy: 0.8151
Epoch 9/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4180 - accuracy: 0.8186
Epoch 10/100
250/250 [==============================] - 0s 1ms/step - loss: 0.4152 - accura

## Confusion Matrix

In [149]:
predictions = predictions > 0.5
cm = confusion_matrix(test_y, predictions)
print(cm)
print('Accuracy: {} %'.format(accuracy_score(test_y, predictions) * 100))

[[1518   77]
 [ 191  214]]
Accuracy: 86.6 %


## Prediction by using input data from user

In [ ]:
credit_score = input('Credit score::')
geography = input('Geography::')
gender = input('Gender::')
age = input('Age::')
tenure = input('Tenure::')
balance = input('Balance::')
no_of_products = input('No of products::')
has_credit_card = input('Has credit card::')
is_active_member = input('Is active member?::')
estimated_salary = input('Estimated salary::')


In [ ]:
gender_encoded = le.transform([gender])
user_sample = [int(credit_score), geography, gender_encoded[0], int(age), int(tenure), int(balance), int(no_of_products), int(has_credit_card), int(is_active_member), int(estimated_salary)]
user_sample = ct.transform([user_sample])
user_sample = sc.transform(user_sample)

In [ ]:
prediction = ann.predict(user_sample) > 0.5
if prediction[0][0] > 0.5:
  print('Exited')
else:
  print('Not Exited')
